In [1]:
import os
import time
import random
import re
import json
import cloudscraper
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

def sanitize_filename(filename):
    """Sanitize the filename by removing invalid characters."""
    return re.sub(r'[<>:"/\\|?*]', '_', filename)

def extract_problems(start_page=4, end_page=5, base_url='https://codeforces.com/problemset/page/'):
    # Directory setup
    problem_text_dir = "./data/problems/problem_statement"
    metadata_dir = "./data/problems/problems_metadata"
    os.makedirs(problem_text_dir, exist_ok=True)
    os.makedirs(metadata_dir, exist_ok=True)

    # To store problem metadata
    problem_metadata_list = []

    # Initialize cloudscraper for CAPTCHA bypass
    scraper = cloudscraper.create_scraper()

    # Selenium WebDriver setup
    chromedriver_path = '/Users/harshpreet/downloads/chromedriver-mac-arm64/chromedriver'
    service = Service(chromedriver_path)
    driver = webdriver.Chrome(service=service)

    def parse_mathjax(element):
        """Extract and format MathJax content."""
        math_content = ""
        mathml = element.get('data-mathml')

        if mathml:
            math_soup = BeautifulSoup(mathml, 'xml')
            for part in math_soup.find_all(['msub', 'msup', 'mn', 'mi', 'mo']):
                if part.name == 'msub':
                    base = part.find('mi').get_text(strip=True)
                    sub = part.find(['mn', 'mi']).get_text(strip=True)
                    math_content += f"{base}_{sub}"
                elif part.name == 'msup':
                    base = part.find(['mn', 'mi']).get_text(strip=True)
                    sup = part.find(['mn', 'mi']).get_text(strip=True)
                    math_content += f"{base}^{sup}"
                else:
                    math_content += part.get_text(strip=True)
        else:
            for part in element.descendants:
                if part.name == 'sup':
                    math_content += f"^{part.get_text(strip=True)}"
                elif part.name == 'sub':
                    math_content += f"_{part.get_text(strip=True)}"
                elif part.string:
                    math_content += part.string.strip()

        return math_content

    for page in range(start_page, end_page + 1):
        page_url = f"{base_url}{page}"
        print(f"Fetching problems from: {page_url}")

        try:
            page_html = scraper.get(page_url).text
            soup = BeautifulSoup(page_html, 'html.parser')
            problem_rows = soup.find_all('td')

            if not problem_rows:
                print(f"No problems found on page {page}.")
                continue

            for row in problem_rows:
                left_div = row.find('div', style="float: left;")
                if left_div and left_div.a:
                    problem_url = f"https://codeforces.com{left_div.a['href']}"
                    problem_title = left_div.a.text.strip()

                    # Extract contest and problem index
                    url_parts = problem_url.split('/')
                    contest_id = url_parts[5]
                    problem_index = url_parts[6]

                    right_div = row.find('div', style="float: right; font-size: 1.1rem;")
                    tags = [tag.text.strip() for tag in right_div.find_all('a')] if right_div else ["Uncategorized"]

                    try:
                        problem_page_html = scraper.get(problem_url).text
                        problem_soup = BeautifulSoup(problem_page_html, 'html.parser')

                        header = problem_soup.find('div', class_='header')
                        time_limit = header.find('div', class_='time-limit').get_text(strip=True) if header else "Unknown"
                        memory_limit = header.find('div', class_='memory-limit').get_text(strip=True) if header else "Unknown"

                        problem_statement = ""
                        problem_section = problem_soup.find('div', class_='problem-statement')

                        if problem_section:
                            for div in problem_section.find_all('div'):
                                for content in div.contents:
                                    if content.name in ['ul', 'ol']:
                                        for li in content.find_all('li'):
                                            text_parts = []
                                            for sub in li.contents:
                                                if sub.name == 'span' and 'mathjax' in sub.get('class', []):
                                                    text_parts.append(parse_mathjax(sub))
                                                elif sub.string:
                                                    text_parts.append(sub.string.strip())
                                            problem_statement += "- " + " ".join(text_parts) + "\n"

                                    elif content.name == 'p':
                                        text_parts = []
                                        for sub in content.contents:
                                            if sub.name == 'span' and 'mathjax' in sub.get('class', []):
                                                text_parts.append(parse_mathjax(sub))
                                            elif sub.string:
                                                text_parts.append(sub.string.strip())
                                        problem_statement += " ".join(text_parts) + "\n\n"

                        # Save problem statement
                        statement_path = os.path.join(problem_text_dir, f"{sanitize_filename(problem_title)}.txt")
                        with open(statement_path, 'w', encoding='utf-8') as text_file:
                            text_file.write(f"Title: {problem_title}\n\n")
                            text_file.write(f"Time Limit: {time_limit}\n")
                            text_file.write(f"Memory Limit: {memory_limit}\n\n")
                            text_file.write(f"Problem Statement:\n{problem_statement}\n")

                        # Save metadata
                        problem_metadata = {
                            "contest_id_problem_index": f"{contest_id}-{problem_index}",
                            "title": problem_title,
                            "url": problem_url,
                            "tags": tags,
                            "time_limit": time_limit,
                            "memory_limit": memory_limit
                        }
                        problem_metadata_list.append(problem_metadata)

                        print(f"Fetched problem: {problem_title}")

                        time.sleep(random.uniform(3, 7))  # Delay to avoid detection

                    except Exception as problem_error:
                        print(f"Error fetching problem at {problem_url}: {problem_error}")

        except Exception as page_error:
            print(f"Error fetching page {page}: {page_error}")

    # Save all metadata to a JSON file
    metadata_path = os.path.join(metadata_dir, "metadata.json")
    with open(metadata_path, 'w', encoding='utf-8') as json_file:
        json.dump(problem_metadata_list, json_file, indent=4)

    driver.quit()

# Execute the function
extract_problems()


Fetching problems from: https://codeforces.com/problemset/page/4
Fetched problem: Shuffle
Fetched problem: Problem Solving
Fetched problem: Message Transmission Error (hard version)
Fetched problem: Message Transmission Error (easy version)
Fetched problem: Three Brothers
Fetched problem: Alternating Sum of Numbers
Fetched problem: Yunli's Subarray Queries (extreme version)
Fetched problem: Yunli's Subarray Queries (hard version)
Fetched problem: Yunli's Subarray Queries (easy version)
Fetched problem: Firefly's Queries
Fetched problem: Klee's SUPER DUPER LARGE Array!!!
Fetched problem: Satyam and Counting
Fetched problem: The Legend of Freya the Frog
Fetched problem: osu!mania
Fetched problem: Minimize!
Fetched problem: Sakurako's Test
Fetched problem: Sakurako's Task
Fetched problem: Sakurako's Box
Fetched problem: Alternating String
Fetched problem: Sakurako's Hobby
Fetched problem: Longest Good Array
Fetched problem: Square or Not
Fetched problem: Sakurako's Exam
Fetched problem: D